# Imports

In [1]:
import re

# Data

In [2]:
bestand = r'.\data\dag6.txt'

with open(bestand, 'r') as file:
    data = (file
            .read()
            .splitlines()
            )
    


# Dag 6.1

In [3]:
def find_initial_coordinate(data):
    row_numnber = 0
    col_number = 0
    for row_number in range(0,len(data)):
        for col_number in range(0,len(data[row_number])):
            if len(re.findall("[<>v^]", data[row_number][col_number])) > 0:
                inital_coordinates = (row_number, col_number)
    return inital_coordinates

In [4]:
def determine_look_direction(data, coordinates):
    direction_looking =''
    if data[coordinates[0]][coordinates[1]] == '^':
        direction_looking = 'up'

    elif data[coordinates[0]][coordinates[1]] == '>':
        direction_looking = 'right'

    elif data[coordinates[0]][coordinates[1]] == 'v':
        direction_looking = 'down'

    elif data[coordinates[0]][coordinates[1]] == '<':
        direction_looking = 'left'

    else:
        raise Exception('coordinates are wrong, no looking position')
    
    new_data = data.copy()
    new_data[coordinates[0]] = re.sub("[<>v^]",'.',new_data[coordinates[0]])

    return new_data, direction_looking

In [5]:
def found_the_escape(data, coordinates, direction_looking):
    number_col = len(data[0])
    number_row = len(data)
    found_escape = False

    if (coordinates[0]==0) & (direction_looking == 'up'):
        found_escape = True
    
    if (coordinates[0]==number_row-1) & (direction_looking == 'down'):
        found_escape = True

    if (coordinates[1]==0) & (direction_looking == 'left'):
        found_escape = True
    
    if (coordinates[1]==number_col-1) & (direction_looking == 'right'):
        found_escape = True
    
    return found_escape
    

    



In [6]:
def define_direction(direction_looking):
    row_direction = 0
    col_direction = 0
    if direction_looking == 'up':
        row_direction = -1
    elif direction_looking == 'down':
        row_direction = 1
    elif direction_looking == 'right':
        col_direction = 1
    elif direction_looking == 'left':
        col_direction = -1

    return row_direction, col_direction

def walk_as_far_as_posible(data, standing_ad_cooridinates, direction_looking, list_of_visits_including_facing, list_of_visits):
    row_direction, col_direction = define_direction(direction_looking)
    row = standing_ad_cooridinates[0]
    col = standing_ad_cooridinates[1]
    boundary_or_end_found = False
    in_loop = False

    i = 1
    j = 1
    while boundary_or_end_found == False:
        next_row = row+row_direction*i
        next_col = col+col_direction*j
        
        if (next_row < 0) | (next_col < 0):
            boundary_or_end_found == True
            break

        try:
            element_next = data[next_row][next_col]
        except:
            boundary_or_end_found == True
            break
        
        if element_next == '.':
            if (next_row, next_col) not in list_of_visits:
                list_of_visits.append((next_row, next_col))
            if (next_row, next_col, direction_looking) not in list_of_visits_including_facing:
                list_of_visits_including_facing.append((next_row, next_col, direction_looking))
            else:
                in_loop = True
                break
            i += 1
            j += 1
        elif element_next == '#':
            boundary_or_end_found = True

    
    new_coordinates = (row+row_direction*(i-1), col+col_direction*(j-1))  

    return list_of_visits_including_facing, list_of_visits, new_coordinates, in_loop

In [7]:
def turn(direction_looking):
    new_direction_looking = ''
    if direction_looking == 'up':
        new_direction_looking = 'right'
    elif direction_looking == 'right':
        new_direction_looking = 'down'
    elif direction_looking == 'down':
        new_direction_looking = 'left'
    elif direction_looking == 'left':
        new_direction_looking = 'up'
    
    return new_direction_looking

In [15]:
def walk_and_count(data):
    standing_ad_cooridinates = find_initial_coordinate(data)
    data, direction_looking = determine_look_direction(data, standing_ad_cooridinates)
    found_escape = False
    list_of_visits = [standing_ad_cooridinates]
    list_of_visits_including_facing =[standing_ad_cooridinates + (direction_looking,)] 
    in_loop = False

    while (found_escape == False | in_loop == False):
        list_of_visits_including_facing, list_of_visits, standing_ad_cooridinates, in_loop = (
            walk_as_far_as_posible(
                data, 
                standing_ad_cooridinates, 
                direction_looking, 
                list_of_visits_including_facing,
                list_of_visits
                )
        )
        found_escape = found_the_escape(data, standing_ad_cooridinates, direction_looking)
        if found_escape == False:
            direction_looking = turn(direction_looking)
    
    number_of_visit_places = len(list_of_visits)
    return number_of_visit_places, in_loop, list_of_visits

In [16]:
number_of_visit_places, in_loop, list_of_visits = walk_and_count(data)
number_of_visit_places

5086

# Dag 6.2

In [10]:
def make_new_data(data,row_number, col_number):
    new_data = data.copy()
    possible = True
    if len(re.findall("[<>v^#]", data[row_number][col_number]))>0:
           possible = False
    new_data[row_number] = new_data[row_number][:col_number]+'#'+data[row_number][col_number+1:]
    return new_data, possible

In [31]:
def count_possible_obstakels(data):
    total_inloop = 0
    for item in list_of_visits:
        row_number = item[0]
        col_number = item[1]
        new_data, possible = make_new_data(data, row_number, col_number)
        if possible:
            number_of_visits, in_loop, list_of_visits_ = walk_and_count(new_data)
            if in_loop:
                total_inloop += 1
    return total_inloop      



In [32]:
count_possible_obstakels(data)

1770